In [45]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier as RFC_
from sklearn.ensemble import RandomForestRegressor as RFR_
from sklearn import metrics

import datetime

w=0

md_c=None
md_r=None
d=datetime.timedelta(365.25*0)
y_active=5
anos_min=5

imp_fert=pd.read_csv('~/git/BD/temp/imp_fert_quanti.csv')

# Seleciona UF e nick
uf_list=['BA']
nick_list_list=[['UREIA_k','MOP_k'],['MAP_k','SAM_k']]

# DF para receber score de cada loop
s=pd.DataFrame()

# Inicia loop sobre as listas de nick
for nick_list in nick_list_list:

    print(nick_list)
    
    # Filtra nick listados
    imp_fert_t=imp_fert[(imp_fert.SG_UF_NCM.astype(str).isin(uf_list))&(imp_fert.nick.astype(str).isin(nick_list))]
    
    # Acerta as tipo das colunas categóricas
    imp_fert_t[['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF']]=imp_fert_t[['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF']].astype('category')

    '''Descartra colunas com apenas conteúdo único
    if len(imp_fert_t.SG_UF_NCM.unique())==1:
        imp_fert_t.drop(columns='SG_UF_NCM',inplace=True)
    if len(imp_fert_t.nick.unique())==1:
        imp_fert_t.drop(columns='nick',inplace=True)'''

    # Inicia loop sobre as datas
    imp_fert_t['date']=pd.to_datetime(imp_fert_t.CO_ANO.astype(str)+'/'+imp_fert_t.CO_MES.astype(str)+'/1')
    date=(imp_fert_t.date.max()-d).floor('d')
    
    # Filtra deixando passar registros menores ou iguais a data do loop
    imp_fert_t2=imp_fert_t[(imp_fert_t.date<=date)]
    
    # Filtra categorias ativas
    imp_fert_t2['date']=pd.to_datetime(imp_fert_t2.CO_ANO.astype(str)+'/'+imp_fert_t2.CO_MES.astype(str)+'/1')
    fim=imp_fert_t2.date.max()
    ini=pd.date_range(end=fim,periods=12*y_active,freq='MS')[0]
    filtra_atividade=imp_fert_t2[imp_fert_t2.date>=ini][imp_fert_t2.select_dtypes(include='category').columns.tolist()].drop_duplicates()
    imp_fert_t2=filtra_atividade.merge(imp_fert_t2)
    
    '''# Limita comprimento do estudo em anos
    ini=pd.date_range(end=fim,periods=12*anos_inclu,freq='MS')[0]
    imp_fert_t2=imp_fert_t2[(imp_fert_t2.date>=ini)]
    
    print('\n',ini,'\n',(imp_fert_t2.date.max()-imp_fert_t2.date.min()).days/365.25)'''
    
    # Confere comprimento mínimo
    if (imp_fert_t2.date.max()-imp_fert_t2.date.min()).days/365.25>anos_min:
        
        ################## Produz a Matriz Esparsa ###########
        # salva datas de início e fim
        ini=imp_fert_t2.date.min()
        fim=imp_fert_t2.date.max()
        # Cria conjunto de anos
        anos_df=pd.DataFrame({'CO_ANO':np.arange(ini.year,fim.year+1)}).assign(key=0)
        
        # Funde anos e categorias (inclusive mês) criando nova tabela ms
        # Note que aqui conjunto de carac que não ocorra em determinado mês não é criado com zero quant. para dar velocidade de processamento
        imp_fert_t2_ms=anos_df.merge(\
            imp_fert_t2[imp_fert_t2.select_dtypes('category').columns.tolist()].assign(key=0).drop_duplicates(),how='outer'
        ).drop(columns='key')
            
        # liminta matriz a datas reais existentes
        imp_fert_t2_ms['date']=pd.to_datetime(imp_fert_t2_ms.CO_ANO.astype(str)+'/'+imp_fert_t2_ms.CO_MES.astype(str)+'/1')
        imp_fert_t2_ms=imp_fert_t2_ms[(imp_fert_t2_ms.date>=ini)&(imp_fert_t2_ms.date<=fim)]
        # funde ms e imp_fert
        imp_fert_t2=imp_fert_t2_ms.merge(imp_fert_t2,how='outer')
        imp_fert_t2['KG_LIQUIDO']=imp_fert_t2[['KG_LIQUIDO']].fillna(0)
        
        ###############################################################################################################
        
        # Diferencia ocorrências reais de não ocorrência
        imp_fert_t2=imp_fert_t2.assign(oc=0)
        imp_fert_t2.loc[imp_fert_t2.KG_LIQUIDO>0,'oc']=1
        
        # Parametriza os algoritmos
        RFC=RFC_(max_depth=md_c)
        RFR=RFR_(max_depth=md_r)
        
        # Adequa medidas de tempo para os algoritmos
        imp_fert_t2.date=imp_fert_t2.date.astype(int)
        imp_fert_t2.drop(columns='CO_ANO',inplace=True)

        # Codifica categóricos
        imp_fert_dumm=pd.get_dummies(imp_fert_t2,prefix_sep='~')

        # Separa treino e teste
        train=imp_fert_dumm[imp_fert_dumm.date< imp_fert_dumm.date.max()]
        test =imp_fert_dumm[imp_fert_dumm.date==imp_fert_dumm.date.max()]
       
        

        # Inicia df para coletar resultados de classificação e regressão do loop
        q=pd.DataFrame()
        r=pd.DataFrame()

        # inicia loop de repetição (para teste de variação dos algorítmos randômicos)
        for x in range(2):

            # Identifica X e y classifier
            X_train=train.drop(columns=['KG_LIQUIDO','oc'])
            y_train=train.oc
            X_test =test.drop(columns=['KG_LIQUIDO','oc'])
            y_test =test.oc
        
            # ******* CLASSIFCAÇÃO *******
            
            # Fit classifier
            RFC.fit(X_train,y_train)
            
            # Predict classifier
            test_pred=X_test.assign(oc_pred=RFC.predict(X_test)).assign(oc=y_test).assign(nick_list=str(nick_list))
            
            # Armazena
            q=pd.concat([q,test_pred])

            
            # ******* REGRESSÃO *******
            
            # Filtra deixando passar registros ocorridos
            train=train[train.oc==1]
            test=test[test.oc==1]
            
            # verifica o comprimento mínimo
            if (len(train)>0)&(len(test)>0):
                
                # Identifica X e y
                X_train=train.drop(columns=['KG_LIQUIDO','oc'])
                y_train=train.KG_LIQUIDO
                X_test =test.drop(columns=['KG_LIQUIDO','oc'])
                y_test =test.KG_LIQUIDO
                
                # Fit regressor
                RFR.fit(X_train,y_train)
                
                # Predict classifier
                test_pred=X_test.assign(kg_pred=RFR.predict(X_test)).assign(KG_LIQUIDO=y_test).assign(nick_list=str(nick_list))
                
                # Armazena
                r=pd.concat([r,test_pred])

        def f_dumm(df_dumm):
            '''Retorna dumm para original'''
            a=pd.from_dummies(df_dumm.select_dtypes(include='bool'),sep='~')
            b=df_dumm.select_dtypes(exclude='bool')
            c=pd.concat([a,b],axis=1)
            return c
        
        q=f_dumm(q)
        r=f_dumm(r)
    
        score=q.merge(r,how='outer')

        score=score.assign()

        s=pd.concat([s,score]).assign(md_c=str(md_c)+'_')\
        .assign(md_r=str(md_r)+'_')\
        .assign(y_active=str(y_active))\
        .assign(anos_min=str(anos_min))\
        .assign(uf_list=str(uf_list))
        
s

['UREIA_k', 'MOP_k']
['MAP_k', 'SAM_k']


,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF,date,oc_pred,oc,nick_list,kg_pred,KG_LIQUIDO,md_c,md_r,y_active,anos_min,uf_list
0,4,MOP_k,BA,149,517800,1711929600000000000,1,1,"['UREIA_k', 'MOP_k']",16691719.20,24694193.0,None_,None_,5,5,['BA']
1,4,MOP_k,BA,149,517800,1711929600000000000,1,1,"['UREIA_k', 'MOP_k']",17884707.25,24694193.0,None_,None_,5,5,['BA']
2,4,MOP_k,BA,149,517800,1711929600000000000,1,1,"['UREIA_k', 'MOP_k']",16691719.20,24694193.0,None_,None_,5,5,['BA']
3,4,MOP_k,BA,149,517800,1711929600000000000,1,1,"['UREIA_k', 'MOP_k']",17884707.25,24694193.0,None_,None_,5,5,['BA']
4,4,MOP_k,BA,149,520100,1711929600000000000,1,0,"['UREIA_k', 'MOP_k']",NaN,NaN,None_,None_,5,5,['BA']
5,4,MOP_k,BA,158,517800,1711929600000000000,0,0,"['UREIA_k', 'MOP_k']",NaN,NaN,None_,None_,5,5,['BA']
6,4,MOP_k,BA,23,517800,1711929600000000000,1,1,"['UREIA_k', 'MOP_k']",14343031.16,1200000.0,None_,None_,5,5,['BA']
7,4,MOP_k,BA,23,517800,1711929600000000000,1,1,"['UREIA_k', 'MOP_k']",14306290.12,1200000.0,None_,None_,5,5,['BA']
8,4,MOP_k,BA,23,517800,1711929600000000000,1,1,"['UREIA_k', 'MOP_k']",14343031.16,1200000.0,None_,None_,5,5,['BA']
9,4,MOP_k,BA,23,517800,1711929600000000000,1,1,"['UREIA_k', 'MOP_k']",14306290.12,1200000.0,None_,None_,5,5,['BA']


In [ ]:
g_list=['date','nick','SG_UF_NCM']
g=[x for x in g_list if x in score.columns]
score=score.groupby(by=g)[['oc','oc_pred','KG_LIQUIDO','kg_pred']].sum().reset_index()

score.loc[score.oc>1,'oc']=1
score.loc[score.oc_pred>1,'oc_pred']=1

score_r=score[(score.oc==1)&(score.oc_pred==1)]
score_r=score_r.assign(pt_pred_real=(score_r.kg_pred/score_r.KG_LIQUIDO*100).astype(int))\
.assign(md_c=str(md_c)+'_').assign(md_r=str(md_r)+'_').assign(d=datetime.timedelta(365.25*0)).assign(y_active=5).assign(anos_inclu=6)\
.assign(uf_list=str(uf_list)).assign(nick_list=str(nick_list))


if w==0:
    score_rb=score_r
    w==1
else:
    score_rb=pd.concat([score_r,score_rb])

score_rb.reset_index(drop=True)

['UREIA_k', 'MOP_k']
['MAP_k', 'SAM_k']


,date,nick,SG_UF_NCM,oc,oc_pred,KG_LIQUIDO,kg_pred,pt_pred_real,md_c,md_r,d,y_active,anos_inclu,uf_list,nick_list
0,1711929600000000000,MAP_k,BA,1,1,238075148.0,1.321486e+08,55,None_,None_,0 days,5,6,['BA'],"['MAP_k', 'SAM_k']"
1,1711929600000000000,SAM_k,BA,1,1,82800000.0,3.813880e+07,46,None_,None_,0 days,5,6,['BA'],"['MAP_k', 'SAM_k']"
